# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96

It is saved in a csv file for you called `cruise_ship_info.csv`. Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis!

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr-exercise').getOrCreate()

In [3]:
data = spark.read.csv('../data/cruise_ship_info.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- Passengers: integer (nullable = true)
 |-- Length: double (nullable = true)
 |-- Cabins: integer (nullable = true)
 |-- Passenger_density: double (nullable = true)
 |-- Crew: integer (nullable = true)



In [5]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       Passengers|           Length|           Cabins|Passenger_density|              Crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+
|  count|      158|        158|               158|               158|              158|              158|              158|              158|               158|
|   mean| Infinity|       null|16.689873417721518| 71.28467088607599| 18.9873417721519|8.130632911392404|9.354430379746836|39.90094936708861| 8.379746835443038|
| stddev|     null|       null| 7.615691058751415|37.229540025907866|9.753645303704317|1.793473548054825|4.503467806583174| 8.63921711391542|3.5915817974968824|
|    min|Adventure|    Azamara|   

In [6]:
data.show(5)

+-----------+-----------+---+-------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|Tonnage|Passengers|Length|Cabins|Passenger_density|Crew|
+-----------+-----------+---+-------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  7| 30.277|         7|  5.94|     4|            42.64|   4|
|      Quest|    Azamara|  7| 30.277|         7|  5.94|     4|            42.64|   4|
|Celebration|   Carnival| 27| 47.262|        15|  7.22|     8|             31.8|   7|
|   Conquest|   Carnival| 12|  110.0|        30|  9.53|    15|            36.99|  20|
|    Destiny|   Carnival| 18|101.353|        27|  8.92|    14|            38.36|  11|
+-----------+-----------+---+-------+----------+------+------+-----------------+----+
only showing top 5 rows



In [7]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [8]:
train_data, test_data = data.randomSplit([.7, .3], seed=True)

In [9]:
from pyspark.ml.feature import StandardScaler, VectorAssembler, StringIndexer

In [10]:
string_indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_indexed', stringOrderType='frequencyDesc')
si_model = string_indexer.fit(train_data)
train_data_indexed = si_model.transform(train_data)
test_data_indexed = si_model.transform(test_data)
train_data_indexed.show(5)

+---------+----------------+---+-------+----------+------+------+-----------------+----+-------------------+
|Ship_name|     Cruise_line|Age|Tonnage|Passengers|Length|Cabins|Passenger_density|Crew|Cruise_line_indexed|
+---------+----------------+---+-------+----------+------+------+-----------------+----+-------------------+
|Adventure| Royal_Caribbean| 13|  138.0|        32|  10.2|    16|            44.32|  12|                1.0|
|  Allegra|           Costa| 22|  28.43|         9|  6.16|     5|            35.19|   5|                4.0|
|Amsterdam|Holland_American| 14|   61.0|        14|   7.8|     7|             44.2|   7|                3.0|
|  Arcadia|             P&O| 10|   85.0|        20|  9.35|    10|            43.19|   9|                8.0|
|  Artemis|             P&O| 30|   45.0|        12|  7.54|     6|             38.2|   6|                8.0|
+---------+----------------+---+-------+----------+------+------+-----------------+----+-------------------+
only showing top 5 

In [11]:
assembler = VectorAssembler(
    inputCols=['Cruise_line_indexed', 'Age','Tonnage','Passengers','Length','Cabins','Passenger_density'],
    outputCol='features'
)
train_data_assembled = assembler.transform(train_data_indexed)
test_data_assembled = assembler.transform(test_data_indexed)
train_data_assembled.show(5)

+---------+----------------+---+-------+----------+------+------+-----------------+----+-------------------+--------------------+
|Ship_name|     Cruise_line|Age|Tonnage|Passengers|Length|Cabins|Passenger_density|Crew|Cruise_line_indexed|            features|
+---------+----------------+---+-------+----------+------+------+-----------------+----+-------------------+--------------------+
|Adventure| Royal_Caribbean| 13|  138.0|        32|  10.2|    16|            44.32|  12|                1.0|[1.0,13.0,138.0,3...|
|  Allegra|           Costa| 22|  28.43|         9|  6.16|     5|            35.19|   5|                4.0|[4.0,22.0,28.43,9...|
|Amsterdam|Holland_American| 14|   61.0|        14|   7.8|     7|             44.2|   7|                3.0|[3.0,14.0,61.0,14...|
|  Arcadia|             P&O| 10|   85.0|        20|  9.35|    10|            43.19|   9|                8.0|[8.0,10.0,85.0,20...|
|  Artemis|             P&O| 30|   45.0|        12|  7.54|     6|             38.2|   6|  

In [12]:
standard_scaler = StandardScaler(inputCol='features', outputCol='features_scaled')
ss_model = standard_scaler.fit(train_data_assembled)
train_data_scaled = ss_model.transform(train_data_assembled)
test_data_scaled = ss_model.transform(test_data_assembled)
train_data_scaled.show(5)


+---------+----------------+---+-------+----------+------+------+-----------------+----+-------------------+--------------------+--------------------+
|Ship_name|     Cruise_line|Age|Tonnage|Passengers|Length|Cabins|Passenger_density|Crew|Cruise_line_indexed|            features|     features_scaled|
+---------+----------------+---+-------+----------+------+------+-----------------+----+-------------------+--------------------+--------------------+
|Adventure| Royal_Caribbean| 13|  138.0|        32|  10.2|    16|            44.32|  12|                1.0|[1.0,13.0,138.0,3...|[0.20998400116536...|
|  Allegra|           Costa| 22|  28.43|         9|  6.16|     5|            35.19|   5|                4.0|[4.0,22.0,28.43,9...|[0.83993600466144...|
|Amsterdam|Holland_American| 14|   61.0|        14|   7.8|     7|             44.2|   7|                3.0|[3.0,14.0,61.0,14...|[0.62995200349608...|
|  Arcadia|             P&O| 10|   85.0|        20|  9.35|    10|            43.19|   9|      

In [13]:
from pyspark.ml.regression import LinearRegression

In [14]:
lr = LinearRegression(featuresCol='features_scaled', labelCol='Crew')

In [15]:
lr_model = lr.fit(train_data_scaled)

In [16]:
test_results = lr_model.evaluate(test_data_scaled)

In [17]:
print(f'RMSE: {test_results.rootMeanSquaredError}')
print(f'R2: {test_results.r2}')

RMSE: 1.6412823020220104
R2: 0.8652294474083793


In [18]:
from pyspark.sql.functions import corr

In [19]:
train_data_indexed.select(
    *[corr('Crew', col) for col in ['Cruise_line_indexed', 'Age','Tonnage','Passengers','Length','Cabins','Passenger_density']]
).show()

+-------------------------------+-------------------+-------------------+----------------------+------------------+------------------+-----------------------------+
|corr(Crew, Cruise_line_indexed)|    corr(Crew, Age)|corr(Crew, Tonnage)|corr(Crew, Passengers)|corr(Crew, Length)|corr(Crew, Cabins)|corr(Crew, Passenger_density)|
+-------------------------------+-------------------+-------------------+----------------------+------------------+------------------+-----------------------------+
|             -0.501334520224341|-0.4771494880978309| 0.9217705612264969|    0.9048274820821302|0.9151415410499385|0.9490561662637791|           -0.238571393722263|
+-------------------------------+-------------------+-------------------+----------------------+------------------+------------------+-----------------------------+



In [20]:
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [21]:
pp = Pipeline(stages=[
    StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_indexed', stringOrderType='frequencyDesc'),
    VectorAssembler(
        inputCols=['Cruise_line_indexed', 'Age','Tonnage','Passengers','Length','Cabins','Passenger_density'],
        outputCol='features'
    ),
    StandardScaler(inputCol='features', outputCol='features_scaled'),
    LinearRegression(featuresCol='features_scaled', labelCol='Crew', predictionCol='Crew_prediction')
])

In [22]:
pp_model = pp.fit(train_data)

In [23]:
test_result = pp_model.transform(test_data)

In [24]:
evaluator = RegressionEvaluator(labelCol='Crew', predictionCol='Crew_prediction')
rmse_score = evaluator.setMetricName('rmse').evaluate(test_result)
r2_score = evaluator.setMetricName('r2').evaluate(test_result)

print(f'RMSE: {rmse_score}')
print(f'R2: {r2_score}')

RMSE: 1.6412823020220104
R2: 0.8652294474083793
